In [1]:
import numpy as np
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fracton/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/fracton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_2018 = getDataframe(2018)
df_2018 = addTokenizedColumnofTitle(df_2018)

In [3]:
df_2018.head()

,Title,Post Type,Number of Comments,Points,Author,tokenized_title
0,Top Hacker News Books in 2017,story,0.0,1,0x54MUR41,"[Top, Hacker News, Books, in]"
1,Beijing enjoys best winter air quality in five...,story,0.0,2,gpetukhov,"[Beijing, enjoy, best, winter, air, quality, i..."
2,Ask HN: Any domain name registrars that don't ...,ask_hn,2.0,1,glockenspielen,"[Ask HN, Any, domain, name, registrars, that, ..."
3,Controversial Therapy Has Led to Death Threats...,story,0.0,2,cpncrunch,"[Controversial, Therapy Has, Led, to, Death Th..."
4,Ruby 3x3 – Ruby 3 Will Be 3 Times Faster – Wha...,story,0.0,2,geraldbauer,"[Ruby Ruby, Will, Be, Times Faster, What]"


In [4]:
AllClasses = list(np.unique(df_2018['Post Type']))
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

## Calculating frequency of each word and given their conditional post type

In [5]:
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)
totalNumberOfWords = getTotalWordCount(words_2018)
words_2018 = obtainDataframeWithClassProbabilities(words_2018, AllClasses, delta, appendClassPrefix)

In [6]:
priorProbabilities = {}
unique, counts = np.unique(df_2018['Post Type'], return_counts=True)
for index in range(len(unique)):
    priorProbabilities[unique[index]] = counts[index]/np.sum(counts)
print(priorProbabilities)

{'ask_hn': 0.04516194251591264, 'poll': 9.025889862481542e-05, 'show_hn': 0.036915889537549505, 'story': 0.917831909047913}


In [7]:
writeModel(words_2018,'model-2018.txt',AllClasses,appendClassPrefix)

In [8]:
words_2018 = renameModelRows(words_2018, AllClasses, appendClassPrefix)

In [9]:
model = [words_2018, priorProbabilities]

In [10]:
sent = "gg fgf fgdfh"
getRandomSentencePrediction(sent,model)

'story'

In [11]:
words_2018.index.values

array(['story', 'ask_hn', 'show_hn', 'poll'], dtype=object)

In [12]:
words_2018.head(10)

,Top,Hacker News,Books,in,Beijing,enjoy,best,winter,air,quality,...,Accomplish,Young Poet,Rainer Maria,Interview Rust,Consultancy Carol,Nichols Goulding,Jake,Goulding,Graphviz Graph,Visualization Software
story,0.000316,0.000070,0.000065,0.018620,0.000029,0.000015,0.000413,0.000023,0.000115,0.000076,...,8.501394e-07,8.501394e-07,8.501394e-07,8.501394e-07,8.501394e-07,8.501394e-07,8.501394e-07,8.501394e-07,8.501394e-07,8.501394e-07
ask_hn,0.000026,0.000195,0.000073,0.006515,0.000002,0.000049,0.001751,0.000007,0.000016,0.000129,...,2.344408e-06,2.344408e-06,2.344408e-06,2.344408e-06,2.344408e-06,2.344408e-06,2.344408e-06,2.344408e-06,2.344408e-06,2.344408e-06
show_hn,0.000054,0.000320,0.000008,0.006478,0.000003,0.000003,0.000257,0.000008,0.000014,0.000054,...,2.827614e-06,2.827614e-06,2.827614e-06,2.827614e-06,2.827614e-06,2.827614e-06,2.827614e-06,2.827614e-06,2.827614e-06,2.827614e-06
poll,0.000005,0.000005,0.000005,0.000014,0.000005,0.000005,0.000005,0.000005,0.000005,0.000005,...,4.784529e-06,4.784529e-06,4.784529e-06,4.784529e-06,4.784529e-06,4.784529e-06,4.784529e-06,4.784529e-06,4.784529e-06,4.784529e-06


## PART 2

In [13]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData)
testData = testData.reset_index()

In [14]:
testData = generateCondClassProb(testData, model)
testData.head()

,index,Title,Post Type,Number of Comments,Points,Author,tokenized_title,ask_hn,poll,show_hn,story
0,276981,The Tech That Was Fixed in 2018 and the Tech T...,story,0.0,3,MagicPropmaker,"[The, Tech, That, Was Fixed, in, and, the, Tec...",-50.822747,-66.932197,-50.048607,-38.445632
1,276982,Why Is the Google Podcasts App Failing So Hard?,story,0.0,4,wenbin,"[Why, Is, the, Google Podcasts, App Failing, S...",-19.816647,-30.168193,-23.236753,-18.183394
2,276983,Doing Dishes Is the Worst,story,0.0,2,paulpauper,"[Doing, Dishes, Is, the]",-16.377811,-24.848032,-18.608992,-13.668253
3,276984,Setting Up a MongoDB Replica Set with Docker a...,story,0.0,1,tugberk,"[Setting, Up, a, MongoDB Replica, Set, with, D...",-44.227409,-60.657794,-41.363934,-35.511676
4,276985,History favors co-operation and non-zero sum g...,story,0.0,3,asimjalis,"[History, favor, and, sum, game]",-22.725299,-30.645314,-22.678649,-17.917274


In [15]:
testData['predicted'] = generatePrediction(testData,AllClasses)
testData.head()   

,index,Title,Post Type,Number of Comments,Points,Author,tokenized_title,ask_hn,poll,show_hn,story,predicted
0,276981,The Tech That Was Fixed in 2018 and the Tech T...,story,0.0,3,MagicPropmaker,"[The, Tech, That, Was Fixed, in, and, the, Tec...",-50.822747,-66.932197,-50.048607,-38.445632,story
1,276982,Why Is the Google Podcasts App Failing So Hard?,story,0.0,4,wenbin,"[Why, Is, the, Google Podcasts, App Failing, S...",-19.816647,-30.168193,-23.236753,-18.183394,story
2,276983,Doing Dishes Is the Worst,story,0.0,2,paulpauper,"[Doing, Dishes, Is, the]",-16.377811,-24.848032,-18.608992,-13.668253,story
3,276984,Setting Up a MongoDB Replica Set with Docker a...,story,0.0,1,tugberk,"[Setting, Up, a, MongoDB Replica, Set, with, D...",-44.227409,-60.657794,-41.363934,-35.511676,story
4,276985,History favors co-operation and non-zero sum g...,story,0.0,3,asimjalis,"[History, favor, and, sum, game]",-22.725299,-30.645314,-22.678649,-17.917274,story


In [16]:
testResults = comparePredictions(testData,AllClasses)
testResults.head()

,Title,predicted,story,ask_hn,show_hn,poll,Post Type,comparision
0,The Tech That Was Fixed in 2018 and the Tech T...,story,-38.445632,-50.822747,-50.048607,-66.932197,story,True
1,Why Is the Google Podcasts App Failing So Hard?,story,-18.183394,-19.816647,-23.236753,-30.168193,story,True
2,Doing Dishes Is the Worst,story,-13.668253,-16.377811,-18.608992,-24.848032,story,True
3,Setting Up a MongoDB Replica Set with Docker a...,story,-35.511676,-44.227409,-41.363934,-60.657794,story,True
4,History favors co-operation and non-zero sum g...,story,-17.917274,-22.725299,-22.678649,-30.645314,story,True


In [17]:
writeDataframe(testResults,'result.txt')

In [18]:
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)

0.9915606894289983


In [19]:
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
print(df_confusion)

predicted  ask_hn  show_hn   story  poll
Post Type                               
ask_hn       5292        1     161     0
poll            2        0       4     0
show_hn        43     4556     304     0
story         439      204  126209     0


In [21]:
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)

predicted  ask_hn  show_hn   story  poll  precision    recall  F1_score
Post Type                                                              
ask_hn       5292        1     161     0   0.916205  0.970297  0.942476
poll            2        0       4     0   0.000000  0.000000  0.000000
show_hn        43     4556     304     0   0.956942  0.929227  0.942881
story         439      204  126209     0   0.996298  0.994931  0.995614


/home/fracton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/fracton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
